In [ ]:
#r "nuget:YahooFinanceApi;" 
#r "../Algos/bin/Debug/QuantLibrary.dll"
using YahooFinanceApi;
using QuantLib;
TSeries data = new();
var history = await Yahoo.GetHistoricalAsync("TQQQ", DateTime.Today.AddDays(-30), DateTime.Now, Period.Daily);

: 

In [ ]:
public class ZLEMA_Series : TSeries
{
    private int _p, _l;
    private bool _NaN;
    private TSeries _data;
    private double _k, _k1m, _lagdata;
    private double _ema, _lastema, _lastlastema;

    public ZLEMA_Series(TSeries source, int period, bool useNaN = true)
    {
        _p = period;
        _l = (int)Math.Ceiling((_p-1) * 0.5);
        _data = source;
        _k = 2.0 / (double)(period + 1);
        _k1m = 1.0 - _k;
        _NaN = useNaN;
        _lastema = _lastlastema = double.NaN;
    }

    public new void Add((System.DateTime t, double v) data, bool update = false)
    {
        if (update) _lastema = _lastlastema;

        _lagdata = _data[_data.Count-1].v + (_data[_data.Count-1].v - _data[Math.Max(_data.Count-1-_l,0)].v);

        _ema = System.Double.IsNaN(_lastema) ? _lagdata : _lagdata * _k + _lastema * _k1m;
        _lastlastema = _lastema;
        _lastema = _ema;

        (System.DateTime t, double v) result = (data.t, (this.Count < _p - 1 && _NaN) ? double.NaN : _ema);
        if (update) base[base.Count - 1] = result; else base.Add(result);
    }

    public void Add(bool update = false)
    {
        this.Add(_data[_data.Count - 1], update);
    }

} 

In [ ]:
SMA_Series sma = new(data, 5, false);
WMA_Series wma = new(data, 5, false);
EMA_Series ema = new(data, 5, false);
HMA_Series hma = new(data, 5, false);
DEMA_Series dema = new(data, 5, false);
TEMA_Series tema = new(data, 5, false);
ZLEMA_Series zlema = new(data, 5, false);

Console.WriteLine($"date\t\t\t Value\t SMA\t WMA\t EMA\t HMA\t DEMA\t TEMA \tZLEMA");
foreach (var i in history) {
    data.Add(i.DateTime,  (double)i.Close);
    sma.Add();
    ema.Add();
    wma.Add();
    hma.Add();
    dema.Add();
    tema.Add();
    zlema.Add();
    Console.WriteLine($"{data[^1].t:yyyy-MM-dd}\t {(double)data:f2}\t {(double)sma:f2}\t {(double)wma:f2}\t {(double)ema:f2}\t {(double)hma:f2}\t {(double)dema:f2}\t {(double)tema:f2}\t {(double)zlema:f2}");
}

date			 Value	 SMA	 WMA	 EMA	 HMA	 DEMA	 TEMA 	ZLEMA
2022-02-18	 50.44	 50.44	 50.44	 50.44	 50.44	 50.44	 50.44	 47.71
2022-02-22	 48.88	 49.66	 49.40	 49.92	 49.75	 49.57	 49.34	 46.67
2022-02-23	 45.14	 48.15	 47.27	 48.33	 46.80	 47.03	 46.25	 44.39
2022-02-24	 49.56	 48.50	 48.19	 48.74	 47.16	 48.15	 48.10	 46.34
2022-02-25	 51.91	 49.19	 49.43	 49.80	 51.21	 50.11	 50.67	 50.45
2022-02-28	 52.46	 49.59	 50.52	 50.68	 53.63	 51.48	 52.19	 52.09
2022-03-01	 50.04	 49.82	 50.67	 50.47	 52.03	 50.86	 51.05	 50.78
2022-03-02	 52.43	 51.28	 51.54	 51.12	 51.49	 51.82	 52.15	 51.32
2022-03-03	 50.24	 51.42	 51.19	 50.83	 51.08	 51.10	 51.03	 51.03
2022-03-04	 48.09	 50.65	 50.08	 49.92	 48.60	 49.49	 48.98	 48.60
2022-03-07	 42.72	 48.70	 47.44	 47.52	 43.56	 45.63	 44.32	 44.13
2022-03-08	 42.13	 47.12	 45.25	 45.72	 40.13	 43.27	 42.02	 41.48
2022-03-09	 46.64	 45.96	 45.09	 46.03	 43.26	 44.60	 44.44	 44.51
2022-03-10	 45.14	 44.94	 44.81	 45.73	 46.04	 44.58	 44.66	 45.72
2022-03-1